## Build-a-thon 2023
Register here https://github.com/vrra/FGAN-Build-a-thon/issues/new/choose

Created: 14 Apr 2022.   
Author: Vishnu Ram OV.   
vishnu.n@ieee.org.   
Licence: available for all purposes under the sun  
but with acknowledgement and citation to "ITU FG AN Build-a-thon"

### Prerequisites
argilla api_url  
argilla api_key  
argilla input_filename  
argilla_workspace  
huggingface hub login token  

### For Colab users:

If you are using colab, add your credentials here and uncomment these  
and skip to step-1 below.

In [ ]:
#api_url="your direct HF Space URL", 
#api_key="your ADMIN_API_KEY"
#input_filename="your input file name"
#argilla_workspace="your argilla workspace"
#vishnuramov_push_to_hf_hub_token="huggingface hub login token"

### For non-colab users:
Create a local file .env with   
api_url="your direct HF Space URL", 
api_key="your ADMIN_API_KEY"  
input_filename="my input filename path"

In [39]:
import argilla as rg

### Read the Argilla HF Space Direct URI, api_key

In [40]:
import os
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [41]:
uri = os.getenv('api_url')
key = os.getenv('api_key')
my_input_filename = os.getenv('input_filename')
my_argilla_workspace=os.getenv('argilla_workspace')
my_push_to_hf_hub_token=os.getenv('vishnuramov_push_to_hf_hub_token')

In [42]:
print(my_argilla_workspace)

admin


In [43]:
rg.init(    
    api_url=uri, 
    api_key=key,
    workspace=my_argilla_workspace
)

### Read the input file 
NOTE- for this notebook we use a downloaded copy of https://www.itu.int/rec/T-REC-Y.Sup71-202207-I

In [44]:
import PyPDF2
pdfFileObj = open(my_input_filename, 'rb')
mypdfReader = PyPDF2.PdfReader(pdfFileObj)

In [45]:
pageObj = mypdfReader.pages[10]
#print(pageObj.extract_text())
meta = mypdfReader.metadata
print(meta.author)
print(meta.creator)
print(meta.producer)
print(meta.subject)
print(meta.title)


FG-AN
Microsoft® Word for Microsoft 365
macOS Version 12.5.1 (Build 21G83) Quartz PDFContext, AppendMode 1.1
None
Use cases for Autonomous Networks


In [46]:
import pandas as pd
dict=   {'author':[],
         'subject': [],
         'title': [],
         'text':[],
         'label':[],
         'metadata':[]
        }
df = pd.DataFrame(dict)

### Configure dataset 
NOTE- we use the labels as 
* use case
* architecture
* poc
* other

In [47]:
settings = rg.TextClassificationSettings(label_schema=["Usecase", "Architecture", "PoC", "Other"])
rg.configure_dataset(name="fgan23allpages", settings=settings)
#Caution: the dataset name will not accept caps letters.

In [48]:
for page in mypdfReader.pages:
    extracted_text=page.extract_text()    
    df.loc[len(df.index)] = [
                            meta.author, 
                            meta.subject, 
                            meta.title, 
                            extracted_text,
                            "", #label
                            ""] #metadata


In [49]:
df.head(3)

,author,subject,title,text,label,metadata
0,FG-AN,None,Use cases for Autonomous Networks,\n \n \n \nI n t e r n a t i o n a l T e l ...,,
1,FG-AN,None,Use cases for Autonomous Networks,\nError! Reference source not found. (2021 -...,,
2,FG-AN,None,Use cases for Autonomous Networks,\nError! Reference source not found. (2021 -...,,


### Placeholder cells for any pandas based pre-processing
At the moment there is no such pandas based pre-processing.

### we will read the df row by row
to account for any pre-processing transformations applied per row
for example per-record metadata

NOTE- this takes more than a minute to complete. 

Grab something.

In [ ]:
for index, row in df.iterrows():
    record = rg.TextClassificationRecord(
                text=row["text"],
                    # Extra information about this record
                metadata={
                        "split": "train"
                    },
                )
    dataset_rg = rg.DatasetForTextClassification([record])
    rg.log(dataset_rg, "fgan23allpages")

### push the dataset to the Hugging Face Hub

In [51]:
from huggingface_hub import login
login(token=my_push_to_hf_hub_token)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /Users/vishnu/.cache/huggingface/token
Login successful


In [52]:
from datasets import load_dataset
dataset_rg = rg.load("fgan23allpages")
dataset_ds = dataset_rg.to_datasets()

#push the dataset to the Hugging Face Hub
dataset_ds.push_to_hub("ITU-T-Build-a-thon-fgan23allpages")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:11<00:00, 11.37s/it]


NOTE- Pages Dataset is available from vishnuramov/ITU-T-Build-a-thon-fgan23allpages

### Example for loading the dataset

In [53]:
from datasets import load_dataset
dataset_ds = load_dataset("vishnuramov/ITU-T-Build-a-thon-fgan23allpages", split="train")
dataset_rg = rg.read_datasets(dataset_ds, task="TextClassification")


[05/10/23 12:37:41] WARNING  WARNING:datasets.builder:Using custom data configuration                ]8;id=243796;file:///opt/homebrew/Caskroom/miniforge/base/envs/myenv38c/lib/python3.8/site-packages/datasets/builder.py\builder.py]8;;\:]8;id=75186;file:///opt/homebrew/Caskroom/miniforge/base/envs/myenv38c/lib/python3.8/site-packages/datasets/builder.py#496\496]8;;\
                             vishnuramov--ITU-T-Build-a-thon-fgan23allpages-594bd03a563e4dad                       

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 806.44it/s]
                                                                     

Dataset parquet downloaded and prepared to /Users/vishnu/.cache/huggingface/datasets/vishnuramov___parquet/vishnuramov--ITU-T-Build-a-thon-fgan23allpages-594bd03a563e4dad/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [54]:
rg.log(dataset_rg, "fgan23allpages_readback")


96 records logged to 
]8;id=717713;https://vishnuramov-build-a-thon-argilla-hf-space-01.hf.space/datasets/admin/fgan23allpages_readback\https://vishnuramov-build-a-thon-argilla-hf-space-01.hf.space/datasets/admin/fgan23allpages_readback]8;;\

BulkResponse(dataset='fgan23allpages_readback', processed=96, failed=0)